In [5]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from scripts.get_news_sentiment import get_news_sentiment
from scripts.get_tweet_sentiment import get_tweet_sentiment

In [40]:
company_names = dict({'Apple':'AAPL', 
                 'Amazon':'AMZN',
                 'Microsoft':'MSFT'})

api_key = 'api'
start_date = '2022-01-01'
end_date = '2022-10-31'




In [3]:
def get_train_data(company:str):
    path = f'../raw_data/{company}.csv'
    return pd.read_csv(path)

def filter_data(df, 
                strt_date:str,
                end_date:str):
    df['Date']=pd.to_datetime(df['Date'])
    df = df[(df['Date'] >= strt_date) & (df['Date'] <= end_date)].reset_index(drop=True)
    return df

In [6]:
df = get_train_data('AAPL')
baseline_df = filter_data(df, start_date, end_date)
news_df = get_news_sentiment('AAPL', start_date, end_date, api_key)
tweet_df = get_tweet_sentiment('AAPL', start_date, end_date, api_key)
train_df = baseline_df.merge(news_df, on='Date').merge(tweet_df, on='Date')


In [63]:
#df_sentiment['Date'] = pd.to_datetime(df_sentiment['Date'])

for keys,values in company_names.items():
    df = get_train_data(values) 
    if values == 'AAPL':
        baseline_df = filter_data(df, start_date, end_date)
        news_df = get_news_sentiment(values, start_date, end_date, api_key)
        tweet_df = get_tweet_sentiment(values, start_date, end_date, api_key)
        train_df = baseline_df.merge(news_df, on='Date').merge(tweet_df, on='Date')
        print(train_df.index[0])
    else:
        baseline_df = filter_data(df, start_date, end_date)
        news_df = get_news_sentiment(values, start_date, end_date, api_key)
        tweet_df = get_tweet_sentiment(values, start_date, end_date, api_key)
        temp_df = baseline_df.merge(news_df, on='Date').merge(tweet_df, on='Date')
        print(temp_df.index[-1])
        #train_df = combine_df(train_df, temp_df)
    
    
    
train_df = baseline_df.merge(news_df, on='Date').merge(tweet_df, on='Date').drop(columns=['Date'])

0
176
176


In [59]:
def combine_df(df1, df2):
    df1_index = df1.index[-1]
    df2_index = len(df2.index)
    print(df1_index)
    print(df2_index)
    df2 = df2.set_index([df1_index + 1, df1_index + df2_index +1 ])
    output = pd.concat([df1, df2])
    return output

In [65]:
def get_X_y_strides(fold: pd.DataFrame, input_length: int, output_length: int, sequence_stride: int):
    '''
    - slides through a `fold` Time Series (2D array) to create sequences of equal
        * `input_length` for X,
        * `output_length` for y,
    using a temporal gap `sequence_stride` between each sequence
    - returns a list of sequences, each as a 2D-array time series
    '''

    X, y = [], []

    for i in range(0, len(fold), sequence_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (i + input_length + output_length) > len(fold):
            break
        X_i = fold.iloc[i:i + input_length, :]
        y_i = fold.iloc[i + input_length:i + input_length + output_length, :][['Close']]
        X.append(X_i)
        y.append(y_i)

    return np.array(X), np.array(y)

In [71]:
X,y = get_X_y_strides(train_df, 10, 1, 1)

In [73]:
y

array([[[301.6000061 ]],

       [[296.36999512]],

       [[296.70999146]],

       [[299.83999634]],

       [[308.76000977]],

       [[313.45999146]],

       [[301.25      ]],

       [[305.94000244]],

       [[300.95001221]],

       [[311.20999146]],

       [[295.        ]],

       [[300.47000122]],

       [[299.5       ]],

       [[290.73001099]],

       [[287.92999268]],

       [[287.72000122]],

       [[280.26998901]],

       [[294.58999634]],

       [[297.30999756]],

       [[298.79000854]],

       [[294.95001221]],

       [[300.19000244]],

       [[295.92001343]],

       [[289.85998535]],

       [[278.91000366]],

       [[288.5       ]],

       [[285.58999634]],

       [[276.44000244]],

       [[287.1499939 ]],

       [[294.39001465]],

       [[295.22000122]],

       [[300.42999268]],

       [[299.16000366]],

       [[304.05999756]],

       [[299.48999023]],

       [[304.1000061 ]],

       [[303.67999268]],

       [[310.70001221]],

       [[315

In [37]:
def get_Xi_yi(
    fold:pd.DataFrame, 
    input_length:int, 
    output_length:int):
    '''
    - given a fold, it returns one sequence (X_i, y_i)
    - with the starting point of the sequence being chosen at random
    '''
    # $CHALLENGIFY_BEGIN
    first_possible_start = 0
    last_possible_start = len(fold) - (input_length + output_length) + 1
    random_start = np.random.randint(first_possible_start, last_possible_start)
    X_i = fold.iloc[random_start:random_start+input_length]    
    y_i = fold.iloc[random_start+input_length:
                  random_start+input_length+output_length][[TARGET]]
    
    return (X_i, y_i)

def get_X_y(
    fold:pd.DataFrame,
    number_of_sequences:int,
    input_length:int,
    output_length:int
):
    # $CHALLENGIFY_BEGIN    
    X, y = [], []

    for i in range(number_of_sequences):
        (Xi, yi) = get_Xi_yi(fold, input_length, output_length)
        X.append(Xi)
        y.append(yi)
        
    return np.array(X), np.array(y)

,0,1,2,3,4,5,6,7,8,9,10
0,2022-03-01,166.160507,167.550003,63520000,170.703506,170.404495,170.404495,31,0.2946,17,0.0412
1,2022-04-01,166.349503,170.438004,70726000,171.399994,167.522003,167.522003,53,0.5572,220,0.0809
2,2022-05-01,164.356995,166.882996,64302000,167.126495,164.356995,164.356995,19,0.4684,175,0.1952
3,2022-06-01,161.936996,163.4505,51958000,164.800003,163.253998,163.253998,46,0.659,373,0.0747
4,2022-07-01,162.031006,163.839005,46606000,165.2435,162.554001,162.554001,36,0.4108,1566,0.107
...,...,...,...,...,...,...,...,...,...,...,...
172,2022-10-18,114.790001,119.059998,65607400,119.519997,116.360001,116.360001,37,0.4689,1349,0.1802
173,2022-10-19,113.220001,114.709999,47198100,116.589996,115.07,115.07,29,0.5404,1068,0.1647
174,2022-10-20,113.510002,113.830002,48795100,118.239998,115.25,115.25,23,0.2823,961,0.2007
175,2022-10-21,114.5,114.790001,55611900,119.589996,119.32,119.32,34,0.476,1217,0.2168


In [ ]:
301
amazon = pd.read_csv('../raw_data/AMZN.csv')
apple = pd.read_csv('../raw_data/AAPL.csv')
micro = pd.read_csv('../raw_data/MSFT.csv')
apple['Date'] = pd.to_datetime(apple['Date'])
amazon['Date'] = pd.to_datetime(amazon['Date'])
micro['Date'] = pd.to_datetime(micro['Date'])

In [ ]:
apple2 = apple[(apple['Date'] > '2021-12-31') & (apple['Date'] < '2022-11-01')]
amazon2 = amazon[(amazon['Date'] > '2021-12-31') & (amazon['Date'] < '2022-11-01')]
micro2 = micro[(micro['Date'] > '2021-12-31') & (micro['Date'] < '2022-11-01')]

In [ ]:
micro2.iloc[:,0:3]

,Date,Low,Open
9026,2022-03-01,329.779999,335.350006
9027,2022-04-01,326.119995,334.829987
9028,2022-05-01,315.980011,325.859985
9029,2022-06-01,311.489990,313.149994
9030,2022-07-01,310.089996,314.149994
...,...,...,...
9225,2022-10-18,235.369995,243.240005
9226,2022-10-19,234.289993,237.039993
9227,2022-10-20,234.869995,235.770004
9228,2022-10-21,234.500000,234.740005


In [ ]:
# Generating Features and Target set
for i in company_names.values():
    path = f'../raw_data/{i}.csv'
    data = pd.read_csv(path)
    my_data = np.genfromtxt(path, delimiter=',')

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
apple = data[data['Date'] > '2021-12-20']

In [ ]:
apple = apple.reset_index(drop=True)

In [ ]:
apple

def train_data_filter(companies:dict, start_date):
    count = 0
    for i in companies.values():
        path = f'../raw_data/{i}.csv'
        data = pd.read_csv(path)
        data['Date'] = pd.to_datetime(data['Date'])
        company_df = data[data['Date'] > '2021-12-31'].reset_index(drop=True)
        features = company_df.to_numpy()
        
        
        
    

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,2021-12-21,169.119995,171.559998,91185900,173.199997,172.990005,172.277100
1,2021-12-22,172.149994,173.039993,92135300,175.860001,175.639999,174.916183
2,2021-12-23,175.270004,175.850006,68356600,176.850006,176.279999,175.553543
3,2021-12-27,177.070007,177.089996,74919600,180.419998,180.330002,179.586868
4,2021-12-28,178.529999,180.160004,79144300,181.330002,179.289993,178.551132
...,...,...,...,...,...,...,...
207,2022-10-18,140.610001,145.490005,99136600,146.699997,143.750000,143.750000
208,2022-10-19,141.500000,141.690002,61758300,144.949997,143.860001,143.860001
209,2022-10-20,142.649994,143.020004,64522000,145.889999,143.389999,143.389999
210,2022-10-21,142.649994,142.869995,86464700,147.850006,147.270004,147.270004


In [ ]:
apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            212 non-null    datetime64[ns]
 1   Low             212 non-null    float64       
 2   Open            212 non-null    float64       
 3   Volume          212 non-null    int64         
 4   High            212 non-null    float64       
 5   Close           212 non-null    float64       
 6   Adjusted Close  212 non-null    float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 11.7 KB


In [ ]:
a = np.array([[1,2,3],[4,5,6]])
a[1][1]

5

In [ ]:
micro.iloc[:,0:3]

,Date,Low,Open
0,1986-03-13,0.088542,0.088542
1,1986-03-14,0.097222,0.097222
2,1986-03-17,0.100694,0.100694
3,1986-03-18,0.098958,0.102431
4,1986-03-19,0.097222,0.099826
...,...,...,...
9225,2022-10-18,235.369995,243.240005
9226,2022-10-19,234.289993,237.039993
9227,2022-10-20,234.869995,235.770004
9228,2022-10-21,234.500000,234.740005


In [ ]:
def get_folds(
    df: pd.DataFrame, 
    fold_length: int,
    fold_stride: int) -> List[pd.DataFrame]:
    '''
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold
    
    Returns a list of folds, each as a DataFrame
    '''
    # $CHALLENGIFY_BEGIN
    folds = []
    for idx in range(0, len(df), fold_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (idx + fold_length) > len(df):
            break
        fold = df.iloc[idx:idx + fold_length, :]
        folds.append(fold)
    return folds